# Load txt to parquet -  ingestion phase

1) Dataframe schema definition<BR> 
2) Read data from azure Row folder to landing zone<BR>
3) Create Audit Log ??

## 1.Setup Folder and Files

### 1.1 List files in relevent storage

In [ ]:
import warnings
warnings.simplefilter(action='ignore') #, category=FutureWarning

In [ ]:
import os
cwd = os.getcwd()
print(cwd)



This is an example of how to list things you need to use the software and how to install them.
* Azure blob storage indigo 
  ```sh
 display(dbutils.fs.ls(srcDataDirRoot))
  ```

In [ ]:
MOUNTPOINT = "c:\BI\GitHub\LearningSparkV2-master\SuppliesML"
SourcePath = os.path.join(MOUNTPOINT, "data")
RawPath    = os.path.join(SourcePath,"Raw")
ResearchPath = os.path.join(SourcePath,"Research")
LandingPath = os.path.join(SourcePath, "Landing")
LoadingPath = os.path.join(SourcePath,  "Loading")
certified_pit = os.path.join(SourcePath,"certified-pit")

In [ ]:
# Define source and destination directories
srcDataDirRoot = RawPath #Root dir for source data
destDataDirRoot = LandingPath #Root dir for consumable data

### 1.2 Define Schema for the file that loaded

In [ ]:
# Required for StructField, StringType, IntegerType, etc.
from pyspark.sql.types import *


# # Space bi inc- sensor data 
ProductionRunSchema = StructType([
  StructField("Product" , StringType() ,True),
  StructField("Machine" , StringType() ,True),
  StructField("FOLDER_PATH" , StringType() ,True),
  StructField("Product_category" , StringType() ,True),
  StructField("Product_eng_name" , StringType() ,True),
  StructField("Series" , StringType() ,True),
  StructField("Product_name_win" , StringType() ,True),
  StructField("Size_Flag" , StringType() ,True),
  StructField("Parameter_Name" , StringType() ,True),
  StructField("Batch" , StringType() ,True),
  StructField("SAMPLE_ID" , FloatType() ,True),
  StructField("SAMPLE_Date" , TimestampType() ,True),
  StructField("SK_Sample_Date" , IntegerType() ,True),
  StructField("Parameter_Critical_Flag" , StringType() ,True),
  StructField("Is_Sample_Deleted_Flg" , StringType() ,True),
  StructField("SAMPLE_Mean" , FloatType() ,True),
  StructField("SAMPLE_stdev" , FloatType() ,True),
  StructField("SAMPLE_Minimum" , FloatType() ,True),
  StructField("SAMPLE_Maximum" , FloatType() ,True),
  StructField("SAMPLE_Median" , FloatType() ,True),
  StructField("Spec_target" , FloatType() ,True),
  StructField("SAMPLE_Size" , FloatType() ,True),
  StructField("LSL" , FloatType() ,True),
  StructField("USL" , FloatType() ,True),
  StructField("SL_enabled" , StringType() ,True),
  StructField("CH_ID" , FloatType() ,True),
  StructField("ETL_DATE" , TimestampType() ,True)
])


# windigo production line
New_Era_Run_Details_Schema = StructType([
StructField("Run_Number" , StringType() ,True),
StructField("Blanket_Serial_Number" , StringType() ,True),  
StructField("Blanket_SEQ_NR" , StringType() ,True),  
StructField("Plant_Id" , IntegerType() ,True),
StructField("Quality_Status_Id" , IntegerType() ,True),
StructField("Body_ID" , StringType() ,True),
StructField("CSL_ID" , StringType() ,True),
StructField("BLK_Quality_Status_Name" , StringType() ,True),
StructField("BLK_Quality_Status_Flag" , IntegerType() ,True),
StructField("Blanket_Legacy_Part_Nr" , StringType() ,True),
StructField("Product_Engineering_Name" , StringType() ,True),
StructField("Source_System_Modified_DateTime" , TimestampType() ,True)
])


# # Customer data - lifespan
Blanket_lifespan_installed_base_Schema = StructType([
#StructField("Fact_PIP_IMPACT_RowID", IntegerType() ,True),
StructField("Blanket_Impact_RowID", StringType() ,True),
StructField("Press_Serial_Number", StringType() ,True),
StructField("BLANKETS_ID", StringType() ,True),
StructField("Replacement_DateTime", StringType() ,True),
StructField("End_User_Code", StringType() ,True),
StructField("Domain", StringType() ,True),
StructField("ROR", StringType() ,True),
StructField("Consumable_Type", StringType() ,True),
StructField("Optimized_Lifespan", IntegerType() ,True),
StructField("Is_Last_Replacement", StringType() ,True),
StructField("Is_Lifespan_Official", StringType() ,True),
StructField("Consumable_Maturity", StringType() ,True),
StructField("DOA_Count", IntegerType() ,True),
StructField("DOP_Count", IntegerType() ,True),
StructField("RowID", IntegerType() ,True),
StructField("Changed_Date_Time", StringType() ,True),
StructField("Replacement_Monthly_Date_Id", IntegerType() ,True),
StructField("ETL_Date", StringType() ,True),
#StructField("Press_Classification", StringType() ,True),
#StructField("Lifespan_Guidelines", DoubleType() ,True),
StructField("Click_Charge", StringType() ,True),
StructField("Ownership", StringType() ,True),
StructField("Product_Number", StringType() ,True),
StructField("Description", StringType() ,True),
StructField("Product_Group", StringType() ,True),
StructField("Press_Group", StringType() ,True),
StructField("Family_type", StringType() ,True),
StructField("Series", StringType() ,True),
StructField("Press_Segment", StringType() ,True),
StructField("Current_SW_Version_ID", StringType() ,True),
StructField("Customer_Name", StringType() ,True),
StructField("Site_Region", StringType() ,True),
StructField("Site_Sub_Region", StringType() ,True),
StructField("Site_Country", StringType() ,True),
#StructField("ETL_Date" , TimestampType() ,True)
])



## 2.Create Spark Session

### 2.1 check warehouse Location

In [ ]:
from os.path import abspath
import pyodbc
warehouse_location = abspath('..\..\..\supplies-warehouse')
print(warehouse_location)

In [ ]:
from os.path import abspath
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pandas as pd
# warehouse_location points to the default location for managed databases and tables
#warehouse_location = abspath('..\..\..\supplies-warehouse')
warehouse_location = abspath('..\..\..\hive_supplies-warehouse')

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example")\
.config("spark.hadoop.hive.metastore.warehouse.dir", warehouse_location)\
.config('spark.driver.extraClassPath', 'C:\Program Files\Microsoft JDBC Driver 10.2 for SQL Server\sqljdbc_10.2\enu\mssql-jdbc-10.2.1.jre8.jar') \
.config('spark.executor.extraClassPath', 'C:\Program Files\Microsoft JDBC Driver 10.2 for SQL Server\sqljdbc_10.2\enu\mssql-jdbc-10.2.1.jre8.jar') \
.config("spark.sql.catalogImplementation","hive") \
.enableHiveSupport() \
.getOrCreate() 


## 3. set data source Connections

### 3.1 Set odbc Connections > user + Password

In [ ]:
#Driver={SQL Server Native Client 11.0};Server=myServerAddress;Database=myDataBase;Uid=myUsername;Pwd=myPassword;

sql_native_space = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=gvs72076.inc.hpicorp.net,2048;"
                        "Database=faomrp;"
                        "Uid=SPACE_BI_IDS;"
                        "Pwd=SPC-IDS!pswd;")
                  
# > windows trusted authication  - when we user password please assign it to "no"
# conn_GABI.close()
                        

In [ ]:
server = 'gvs72076.inc.hpicorp.net,2048' 
database = 'faomrp' 
username = 'SPACE_BI_IDS' 
password = 'SPC-IDS!pswd' 
odbc_SPACE_BI = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
print(odbc_SPACE_BI,database)

server = '106775-2s-ag1.corp.hpicloud.net,2048' 
database = 'GABI-I' 
username = 'GABI_STG_RW_1' 
password = 'change_pw_first_login2020$' 
odbc_SPACE_BI = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
print(odbc_SPACE_BI,database)

server = 'gvs72069.inc.hpicorp.net,2048' 
database = 'GABI-P' 
username = 'GABI_STG_RW_1' 
password = 'fT7_E*4U6k_m' 
odbc_GABI_P = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
print(odbc_SPACE_BI,database)


server = 'WIN-OA97GH5SPHI.CORP.HPICLOUD.NET,1433' # 172.20.206.100
database = 'Hubble_DWH_ITG' 
username = 'Tableau_Itg' 
password = 'Tab_itg4' 
odbc_Hubble_ITG = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
print(odbc_Hubble_ITG,database)



### 3.2 Set odbc Connections > user authntication

In [145]:
import pyodbc
Driver="{ODBC Driver 17 for SQL Server}"
server="172.20.206.100,1433"
database="HUBBLE_DWH_ITG"
trusted_connection="yes"
odbc_Hubble_ITG = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';Database='+database+';trusted_connection='+trusted_connection)
print(odbc_Hubble_ITG)

In [ ]:
#Driver={SQL Server Native Client 11.0};Server=myServerAddress;Database=myDataBase;Uid=myUsername;Pwd=myPassword;
import pyodbc
sql_native_Hubble_ITG = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=172.20.206.100,1433;"
                        "Database=HUBBLE_DWH_ITG;"
                        "Uid=Tableau_Itg;"
                        "Pwd=Tab_itg4;")
                  
# > windows trusted authication  - when we user password please assign it to "no"
# conn_GABI.close()

### 3.3 Set odbc Connections > user + passward

sqlsUrl = 'jdbc:sqlserver://172.20.206.100:1433;encrypt=true;databaseName=HUBBLE_DWH_ITG;integratedSecurity=true;trustServerCertificate=true;'

>> there is no integrated security 

### 3.4 set jdbc Connections

In [ ]:
#sqlsUrl = 'jdbc:sqlserver://gvs72069.inc.hpicorp.net:2048;encrypt=true;database=GABI-P;user=GABI_STG_RW_1;password=fT7_E*4U6k_m;trustServerCertificate=true;'
JdbcsqlsUrl_space = 'jdbc:sqlserver://gvs72076.inc.hpicorp.net:2048;encrypt=true;database=faomrp;user=SPACE_BI_IDS;password=SPC-IDS!pswd;trustServerCertificate=true;'

In [ ]:
JdbcsqlsUrl_NewEra = 'jdbc:sqlserver://gvs72069.inc.hpicorp.net:2048;encrypt=true;database=GABI-P;user=GABI_STG_RW_1;password=fT7_E*4U6k_m;trustServerCertificate=true;'

In [ ]:
JdbcsqlsUrl_blanketslifespan = 'jdbc:sqlserver://WIN-OA97GH5SPHI.CORP.HPICLOUD.NET:1433;encrypt=true;database=HUBBLE_DWH_ITG;user=Tableau_Itg;password=Tab_itg4;trustServerCertificate=true;'

## 5. List of queires

## 5.1 Load with JDBC

#### 5.1.1 Load from New Era

In [ ]:
import time
from datetime import datetime, timedelta,date
# Build Dynamic Query per yearmonth filter
def Loaddata(YearMonth,StartYear,querypath):
    
    #YearMonth ="201903"
    Year=YearMonth[0:4]
    with open(querypath) as f: query_file = f.read()
    sql_string = query_file.replace("{YearMonth}",str(YearMonth)).replace("{StartYear}",str(StartYear))
    dynamic_sql = f"""{sql_string}"""
    return dynamic_sql


# Read Query from Source >  Return DATAFRAME per yearmonth filter
def sprak_read_df(Query,schema):
    
    startTime = time.time()
    spark_read_df = spark.read.format('jdbc') \
    .option('url',JdbcsqlsUrl_NewEra) \
    .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver')\
    .option('query', Query) \
    .schema(schema) \
    .load()
    executionTime = (time.time() - startTime)
    #print('Execution time in seconds: ' + str(round(executionTime,2)))
    return spark_read_df


def WriteToParqurtFile(Dataframe,Zone,Table_Folder,Year,YearMonth):
    Dataframe.coalesce(8).write.parquet(LandingPath + Table_Folder +str(Year)+"/"+str(YearMonth),mode="overwrite") 
    executionTime = (time.time() - startTime)

min_query_date= '2020-11-01'  
StartYear = "2018"
Zone = LandingPath
Table_Folder = "/New_Era/New_era_details/"
Table = "NewEra"
QueryPath = "IngestQueries/NewEra.sql"
startPipelineTime =  time.time()

#for beg in pd.date_range(min_query_date,date.today().strftime("%Y-%m-01"), freq='MS'): # date.today() date.today().strftime("%Y-%m-01")
for beg in pd.date_range('2020-10-01','2022-11-01', freq='MS'): # date.today()
    startTime = time.time()
    Yearmonth = beg.strftime("%Y%m")
    Year=Yearmonth[0:4]                                     
    dynamic_sql = Loaddata(Yearmonth,StartYear,QueryPath)   
    
    Dataframe = sprak_read_df(dynamic_sql,New_Era_Run_Details_Schema)
    #WriteToParqurtFile(Dataframe,Zone,Table_Folder,Year,Yearmonth)
    
    executionTime = (time.time() - startTime)
    print(f'Execution time in seconds for: Table '+Table+' > YearMonth: '+ Yearmonth +"  is: "+ str(round(executionTime,2))+ "   numberofrow:  " + str(Dataframe.count()))
executionTime = (time.time() - startPipelineTime)
print(f'total Execution time in seconds for: Table '+Table+' > all yearmonth is: '+ str(round(executionTime,2)))   
    

### 5.1.2.1 read & write & Test Count / Show() all NEW ERA yearmonth file to one parquet file

In [136]:
New_era_details_dfs = spark.read.format("parquet").load(LandingPath + "/New_Era/New_era_details/*/*") # try to read empty

In [137]:
New_era_details_dfs.createOrReplaceTempView("ld_New_Eras")

In [138]:
ld_new_era_Yield_df = spark.sql(
"select Run_Number,CAST (Sum(BLK_Quality_Status_Flag)/Count(Blanket_SEQ_NR) AS Decimal(5,2)) AS Yield  from ld_New_Eras Group by Run_Number")

In [139]:
ld_New_Era =New_era_details_dfs.join(ld_new_era_Yield_df,New_era_details_dfs.Run_Number ==  ld_new_era_Yield_df.Run_Number,"left").drop(ld_new_era_Yield_df.Run_Number)

In [140]:
ld_New_Era_new = spark.sql("select *, Year(Source_System_Modified_DateTime)*100+Month(Source_System_Modified_DateTime) As YearMonth from ld_New_Era")


In [ ]:
import time
startTime = time.time()
New_era_details_dfs.coalesce(1).write.parquet(LandingPath + "/New_Era/New_era_details_Full/New_era_details_Full.parquet",mode="overwrite") #e")#.saveAsTable("ld_blanket_lifespan_run_extended")# = spark.read.format("parquet").load(SourcePath + '/test')
executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(round(executionTime,2)))


#### 5.1.2 Load from SpaceBI > production run jdbc

In [ ]:
import time
from datetime import datetime, timedelta,date

# Build Dynamic Query per yearmonth filter
def Loaddata(YearMonth,StartYear,querypath):
    
    #YearMonth ="201903"
    Year=YearMonth[0:4]
    with open(querypath) as f: query_file = f.read()
    sql_string = query_file.replace("{YearMonth}",str(YearMonth)).replace("{StartYear}",str(StartYear)).replace("{FolderPath}",str("'"+FolderPath+"'")).replace("{ProductName}",str( "'"+ProductName+"'"))
    dynamic_sql = f"""{sql_string}"""
    #print(dynamic_sql)
    #print(YearMonth)
    return dynamic_sql


# Read Query from Source >  Return DATAFRAME per yearmonth filter
def sprak_read_df(Query,schema,conn):
    
    startTime = time.time()
    spark_read_df = spark.read.format('jdbc') \
    .option('url',conn) \
    .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver')\
    .option('query', Query) \
    .schema(schema) \
    .load()
    executionTime = (time.time() - startTime)
    #print('Execution time in seconds: ' + str(round(executionTime,2)))
    return spark_read_df

def pandas_read_df(dynamic_sql,sql_native_space):
    fmt='%Y%m%d %H:%M:%S'             
    pandas_read_pdf = pd.read_sql_query(sql = dynamic_sql,\
                                        con = sql_native_space,\
                                        parse_dates={'SAMPLE_Date':fmt,'ETL_DATE':fmt}\
                                       )
                                        #parse_dates={'Date_of_creation':fmt, \ 
                                        #'Date_of_termination':fmt})  
    return pandas_read_pdf

def WriteToParqurtFile(Dataframe,Zone,Table_Folder,Year,Yearmonth):
    spark_read_df=spark.createDataFrame(Dataframe,ProductionRunSchema) #ProductionRunSchema
    spark_read_df.coalesce(1).write.parquet(LandingPath + Table_Folder +str(Year)+"/"+str(Yearmonth),mode="overwrite") 
    executionTime = (time.time() - startTime)
#Blanket_lifespan_installed_base_Schema

# def PandasWriteToParqurtFile(Dataframe,Zone,Table_Folder,Year,YearMonth):
#     Dataframe.to_parquet(LandingPath + Table_Folder +str(Year)+"/"+str(YearMonth),mode="overwrite") 
#     executionTime = (time.time() - startTime)
# Blanket_lifespan_installed_base_Schema



ProductList = {}
ProductList['Gemini3'] = {'StartYearMonth': '2020-10-01', 'FullPath': 'New Era+AFL+WSM-> Gemini 3','FolderOutput':'Gemini', 'ProductName': 'GEMINI3'}
ProductList['RotemSP'] = {'StartYearMonth': '2020-11-01', 'FullPath': 'New Era+AFL+WSM-> NewEra-SpeedUp-> Rotem SP','FolderOutput':'RotemSP', 'ProductName': 'ROTEM_SP'}
ProductList['IrisPlusSP'] = {'StartYearMonth': '2020-10-01', 'FullPath': 'New Era+AFL+WSM-> NewEra-SpeedUp-> Iris Plus SP','FolderOutput':'IrisPlusSP', 'ProductName': 'IRIS PLUS_SP'}
ProductList['Timna2'] = {'StartYearMonth': '2020-10-01', 'FullPath': 'New Era+AFL+WSM-> Timna 2','FolderOutput':'Timna2', 'ProductName': 'TIMNA 2'}


### Production run queries variables
## **************************************
# find Min Date To load Data
querypath = "IngestQueries/ProductionLine.sql"
min_date = '2019-01-01'
StartYear = "2018"
FolderPath = ProductList['IrisPlusSP']['FullPath']
ProductName = ProductList['IrisPlusSP']['ProductName']
with open(querypath) as f: query_file = f.read()
sql_string = query_file.replace("{StartYear}",str(StartYear)).replace("{FolderPath}",str("'"+FolderPath+"'")).replace("{ProductName}",str( "'"+ProductName+"'"))
dynamic_sql = f"""{sql_string}"""
#print(str("'"+FolderPath+"'"))
min_query_date_pdf = pd.read_sql_query(sql = dynamic_sql,  con = sql_native_space)  
start_date = min_query_date_pdf['MinYearMonth'][0].strftime("%Y-%m-01")
min_query_date = str(max(min_date,start_date))
## **************************************

min_query_date= '2022-12-01'
    
Zone = LandingPath
product = "IrisPlusSP"   # Iris 6234  '2022-11-01'  # rptem Count before inner 2460
Table_Folder = "/ProductionRun/ProductionRun_hist/"+product+"/"
#Table_Folder = "/New_Era/New_era_details/"
Table = "ProductionRun"
QueryPath = "IngestQueries/ProductionRun.sql"
startPipelineTime =  time.time()
for beg in pd.date_range(min_query_date,date.today().strftime("%Y-%m-01"), freq='MS'): # date.today() date.today().strftime("%Y-%m-01")
    startTime = time.time()
    Yearmonth = beg.strftime("%Y%m")
    Year=Yearmonth[0:4]                                     
    dynamic_sql = Loaddata(Yearmonth,StartYear,QueryPath)     
    print('finish dynmic',Yearmonth)
    Dataframe = pandas_read_df(dynamic_sql,sql_native_space)
    print('finish READ QUERY')
    WriteToParqurtFile(Dataframe,Zone,Table_Folder,Year,Yearmonth)
    
    executionTime = (time.time() - startTime)
    print(f'Execution time in seconds for: Table '+Table+' > YearMonth: '+ Yearmonth +"  is: "+ str(round(executionTime,2))+ "   numberofrow:  " + str(len(Dataframe.index)))

executionTime = (time.time() - startPipelineTime)
print(f'{datetime.now()} +   total Execution time in seconds for: Table '+Table+' > all yearmonth is: '+ str(round(executionTime,2)))   



In [ ]:
### 5.1.2.1 read & write & Test Count / Show() all space BI yearmonth file to one parquet file

In [123]:
# Parameters_run_level_dfs = spark.read.format("parquet").load(LandingPath + "/ProductionRun/ProductionRun_hist/*/*/*")
Parameters_run_level_dfs = spark.read.format("parquet").load(LandingPath + "/ProductionRun/ProductionRun_hist/*/*/*/*") # try to read empty


In [125]:
Parameters_run_level_dfs.createOrReplaceTempView("ld_Parameters_run_level_dfs")

In [127]:
ld_Parameters_run_level_dfs_new = spark.sql("select *, Year(SAMPLE_Date)*100+Month(SAMPLE_Date) As YearMonth from ld_Parameters_run_level_dfs")


In [ ]:
ld_Parameters_run_level_dfs_new.createOrReplaceTempView("ld_Parameters_run_level_dfs_new")

In [129]:
startTime = time.time()
ld_Parameters_run_level_dfs_new.coalesce(1).write.parquet(LandingPath + "/ProductionRun/ProductionRun_Full/ProductionRun_Full_YM.parquet",mode="overwrite") #e")#.saveAsTable("ld_blanket_lifespan_run_extended")# = spark.read.format("parquet").load(SourcePath + '/test')
executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(round(executionTime,2)))
Parameters_run_level_dfs.count()

Execution time in seconds: 6.96


494670

#### 5.1.2.1 TEST Load from SpaceBI > production run jdbc



#### Load With pyodbc - Blankets Lifespan

In [147]:
import time
from datetime import datetime, timedelta,date
from pyspark.sql.types import StructType, IntegerType, StringType, LongType
import pandas as pd
from pyspark.sql.functions import col

# Build Dynamic Query per yearmonth filter
def Loaddata(YearMonth,StartYear,querypath):
    
    #YearMonth ="201903"
    Year=YearMonth[0:4]
    with open(querypath) as f: query_file = f.read()
    sql_string = query_file.replace("{YearMonth}",str(YearMonth)).replace("{StartYear}",str(StartYear))
    dynamic_sql = f"""{sql_string}"""
    #print(dynamic_sql)
    #print(YearMonth)
    return dynamic_sql


convert_dict = {#'Blanket_Impact_RowID': str,
                'Optimized_Lifespan': int,
                #'Replacement_DateTime': str
                }
 


# Read Query from Source >  Return DATAFRAME per yearmonth filter
def pandas_read_df(query,conn):   
    startTime = time.time()
    Pareto_pdf = pd.read_sql_query(sql = query, con = conn)     
    print("after_read_sql_query")
    
    Pareto_pdf = Pareto_pdf.astype(convert_dict)
    #Pareto_pdf = Pareto_pdf.tz_localize(tz="Asia/Jerusalem", nonexistent=pd.Timedelta('1H'))
    Pareto_pdf['ETL_Date'] = pd.to_datetime(Pareto_pdf['ETL_Date']) \
.dt.tz_localize('Asia/Jerusalem',nonexistent=pd.Timedelta('1H'))
    print("after_convert_in_pandas")
    return Pareto_pdf


def WriteToParqurtFile(Dataframe,Zone,Table_Folder,Year,YearMonth):
    #Dataframe = Dateframe['Blanket_Impact_RowID'].astype('int')
    spark_read_df_temp =spark.createDataFrame(Dataframe,Blanket_lifespan_installed_base_Schema)
    spark_read_df_temp.coalesce(8).write.parquet(LandingPath + Table_Folder +str(Year)+"/"+str(YearMonth),mode="overwrite") 
    executionTime = (time.time() - startTime)

    
Zone = LandingPath
#Table_Folder = "/Blanket_lifespan/Blanket_lifespan_Hist_New/"
Table_Folder = "/Blanket_lifespan/Blanket_lifespan_Hist_Test_pandas_n/"
Table = "Blanketslifespan"
QueryPath = "IngestQueries/Blanketslifespan.sql"
#Conn = odbc_Hubble_ITG
StartYear = "2019"
startPipelineTime =  time.time()
for beg in pd.date_range('2020-12-01','2022-12-01', freq='MS'): # date.today()
    startTime = time.time()
    YearMonth = beg.strftime("%Y%m")
    Year=YearMonth[0:4]                                     
    dynamic_sql = Loaddata(YearMonth,StartYear,QueryPath)  
    #print(dynamic_sql)
    Dataframe = pandas_read_df(dynamic_sql,odbc_Hubble_ITG  ) #sql_native_Hubble_ITG
    Dataframe.to_parquet(LandingPath + Table_Folder +"/"+str(YearMonth)+".parquet")
    #WriteToParqurtFile(Dataframe,Zone,Table_Folder,Year,YearMonth)
    executionTime = (time.time() - startTime)
    print(f'Execution time in seconds for: Table '+Table+' > YearMonth: '+ YearMonth +"  is: "+ str(round(executionTime,2))+ "   numberofrow:  " + str(len(Dataframe.index)))
    
executionTime = (time.time() - startPipelineTime)
print(f'{datetime.now()} total Execution time in seconds for: Table '+Table+' > all yearmonth is: '+ str(round(executionTime,2))) 




after_read_sql_query
after_convert_in_pandas
Execution time in seconds for: Table Blanketslifespan > YearMonth: 202012  is: 210.57   numberofrow:  69502
after_read_sql_query
after_convert_in_pandas
Execution time in seconds for: Table Blanketslifespan > YearMonth: 202101  is: 77.29   numberofrow:  59427


In [114]:
Blanketslifespan = spark.read.format("parquet").load(LandingPath + Table_Folder +"/*")

In [133]:
Blanketslifespan.coalesce(1).write.parquet(LandingPath + "/Blanket_lifespan/Blanket_lifespan_hist_PM.parquet",mode="overwrite")